In [290]:
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from cmath import phase
import scipy.special as spe
import numpy as np
pio.renderers.default='iframe'

def ftheta(l,m,theta):
    C=np.sqrt((2*l+1)*spe.factorial(l-m)/(4*np.pi)/spe.factorial(l+m))
    legendre=spe.lpmv(m,l,np.cos(theta))
    return C*legendre

def fphi(m, phi):
    return np.exp(1j*m*phi)

class spherical_harmonics:
    
    def __init__(self,l,m):
        
        #definition of class atributes
        self.l=l
        self.m=m
        self.theta=np.linspace(0, np.pi, 50)
        self.phi=np.linspace(0, 2*np.pi, 50)
       
        
        #we will also save the computed results of ftheta() and fphi()
        C1=ftheta(self.l,self.m,self.theta)
        C2=fphi(self.m,self.phi)
        
        self.theta_vals=C1
        self.phi_vals=C2
        
        #outer product, its absolute value and the square of its absolute
        #value of the two arrays above will 
        #also be stored for future purposes
        self.out_product=np.outer(C2, C1)
        self.module=np.abs(self.out_product)
        self.prob=np.abs(self.out_product)**2
        
        #Plot constants, only contain normalized grid of spherical coordinates
        self.x_plot=np.outer(np.cos(self.phi), np.sin(self.theta))
        self.y_plot=np.outer(np.sin(self.phi), np.sin(self.theta))
        self.z_plot=np.outer(np.ones_like(self.phi), np.cos(self.theta))
        #self.phi_plot = np.outer(np.cos(self.phase),np.ones_like(self.theta))
        
        
        #these loops are meant to calculate the proper phase of every single point. 
        #it its calculated recursiverly due to the need of having opposite phases
        if (m!=0):
            self.phi_plot=np.zeros([len(self.z_plot),len(self.z_plot)])
        
            for i in range(len(self.z_plot)):
                a=0-np.pi*i
                for j in range(len(self.z_plot)):
                
                    if (np.sign(self.z_plot[j,i])<0):
                        self.phi_plot[j,i]=np.cos(self.phi[j]+np.pi)
                
                    if (np.sign(self.z_plot[j,i])>0):
                        self.phi_plot[j,i]=np.cos(self.phi[j])
                        
        if (m==0):
            self.phi_plot = np.outer(np.ones_like(self.theta), np.cos(self.phi))
            
    def plot_prob(self):
        
        prob_plotx=self.prob*self.x_plot
        prob_ploty=self.prob*self.y_plot
        prob_plotz=self.prob*self.z_plot
        
        fig = go.Figure(data=[go.Surface(x=prob_plotx, y=prob_ploty, z=prob_plotz, 
                                         surfacecolor=self.prob, colorscale='Oranges')])
  
        fig.update_traces(contours_z=dict(
               show=True, usecolormap=True,
               highlightcolor="limegreen",
               project_z=True))
        
        #this variable is used to dynamically adjust the graph initial zoom
        aux1=np.max(self.z_plot)
        
        #starting zoom of the graph. Relies on the l variable to adjust the graph
        camera = dict(
               eye=dict(x=1.25+aux1*(self.l-np.abs(self.m)), y=1.25+aux1*(self.l-np.abs(self.m)), z=1.25)
        )    
    
        fig.update_layout(scene_aspectmode='data', scene_camera=camera)
  
        fig.show()
    
            
    def plot_phase(self):
        
        phase_plotx=self.module*self.x_plot
        phase_ploty=self.module*self.y_plot
        phase_plotz=self.module*self.z_plot
        
        
        
        fig = go.Figure(data=[go.Surface(x=phase_plotx, y=phase_ploty, z=phase_plotz,
                                         surfacecolor=self.phi_plot,
                                         colorscale='HSV'
                                        )])

        fig.update_traces(contours_z=dict(
              show=True, usecolormap=True,
              highlightcolor="limegreen",
              project_z=True))
        
        #this variable is used to dynamically adjust the graph initial zoom
        aux1=np.max(self.z_plot)
        
        #starting zoom of the graph. Relies on the l variable to adjust the graph
        camera1 = dict(
               eye=dict(x=1.25+aux1*(self.l-np.abs(self.m)), y=1.25+aux1*(self.l-np.abs(self.m)), z=1.25)
        )
        
        fig.update_layout(scene_aspectmode='data', scene_camera=camera1)
  
        fig.show()

In [292]:
spherical_harmonics(6,2).plot_phase()

In [244]:
spherical_harmonics(3,1).plot_prob()

In [236]:
import numpy.linalg as linalg
import matplotlib
a=spherical_harmonics(4,2).prob[0]
b=spherical_harmonics(4,2).phi_plot
print(a)
print(a/linalg.norm(a))
print(b)

[0.         0.04110671 0.18451004 0.0354077  0.06551323 0.06551323
 0.0354077  0.18451004 0.04110671 0.        ]
[0.         0.14306419 0.64215267 0.12322987 0.22800654 0.22800654
 0.12322987 0.64215267 0.14306419 0.        ]
[[ 1.          1.          1.          1.          1.         -1.
  -1.         -1.         -1.         -1.        ]
 [ 0.76604444  0.76604444  0.76604444  0.76604444  0.76604444 -0.76604444
  -0.76604444 -0.76604444 -0.76604444 -0.76604444]
 [ 0.17364818  0.17364818  0.17364818  0.17364818  0.17364818 -0.17364818
  -0.17364818 -0.17364818 -0.17364818 -0.17364818]
 [-0.5        -0.5        -0.5        -0.5        -0.5         0.5
   0.5         0.5         0.5         0.5       ]
 [-0.93969262 -0.93969262 -0.93969262 -0.93969262 -0.93969262  0.93969262
   0.93969262  0.93969262  0.93969262  0.93969262]
 [-0.93969262 -0.93969262 -0.93969262 -0.93969262 -0.93969262  0.93969262
   0.93969262  0.93969262  0.93969262  0.93969262]
 [-0.5        -0.5        -0.5        -

In [245]:
print('HSV')

HSV


In [249]:
print(spherical_harmonics(3,1).phi_plot)

[[ 1.          1.          1.         ... -1.         -1.
  -1.        ]
 [ 0.99179001  0.99179001  0.99179001 ... -0.99179001 -0.99179001
  -0.99179001]
 [ 0.96729486  0.96729486  0.96729486 ... -0.96729486 -0.96729486
  -0.96729486]
 ...
 [ 0.96729486  0.96729486  0.96729486 ... -0.96729486 -0.96729486
  -0.96729486]
 [ 0.99179001  0.99179001  0.99179001 ... -0.99179001 -0.99179001
  -0.99179001]
 [ 1.          1.          1.         ... -1.         -1.
  -1.        ]]
